# EventVestor: Legal and Regulatory

In this notebook, we'll take a look at EventVestor's *Legal and Regulatory* dataset, available on the [Quantopian Store](https://www.quantopian.com/store). This dataset spans January 01, 2007 through the current day, and documents major legal and regulatory events affecting publicly traded companies.

### Blaze
Before we dig into the data, we want to tell you about how  you generally access Quantopian Store data sets. These datasets are available through an API service known as [Blaze](http://blaze.pydata.org). Blaze provides the Quantopian user with a convenient interface to access very large datasets.

Blaze provides an important function for accessing these datasets. Some of these sets are many millions of records. Bringing that data directly into Quantopian Research directly just is not viable. So Blaze allows us to provide a simple querying interface and shift the burden over to the server side.

It is common to use Blaze to reduce your dataset in size, convert it over to Pandas and then to use Pandas for further computation, manipulation and visualization.

Helpful links:
* [Query building for Blaze](http://blaze.pydata.org/en/latest/queries.html)
* [Pandas-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-pandas.html)
* [SQL-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-sql.html).

Once you've limited the size of your Blaze object, you can convert it to a Pandas DataFrames using:
> `from odo import odo`  
> `odo(expr, pandas.DataFrame)`

### Free samples and limits
One other key caveat: we limit the number of results returned from any given expression to 10,000 to protect against runaway memory usage. To be clear, you have access to all the data server side. We are limiting the size of the responses back from Blaze.

There is a *free* version of this dataset as well as a paid one. The free one includes about three years of historical data, though not up to the current day.

With preamble in place, let's get started:

In [2]:
# import the dataset
from quantopian.interactive.data.eventvestor import legal_and_regulatory
# or if you want to import the free dataset, use:
# from quantopian.interactive.data.eventvestor import legal_and_regulatory_free

# import data operations
from odo import odo
# import other libraries we will use
import pandas as pd

In [3]:
# Let's use blaze to understand the data a bit using Blaze dshape()
legal_and_regulatory.dshape

dshape("""var * {
  event_id: ?float64,
  asof_date: datetime,
  trade_date: ?datetime,
  symbol: ?string,
  event_type: ?string,
  event_headline: ?string,
  legal_amount: ?float64,
  legal_units: ?string,
  legal_entity: ?string,
  event_rating: ?float64,
  timestamp: datetime,
  sid: ?int64
  }""")

In [4]:
# And how many rows are there?
# N.B. we're using a Blaze function to do this, not len()
legal_and_regulatory.count()

8180

In [5]:
# Let's see what the data looks like. We'll grab the first three rows.
legal_and_regulatory[:3]

,event_id,asof_date,trade_date,symbol,event_type,event_headline,legal_amount,legal_units,legal_entity,event_rating,timestamp,sid
0,77848,2007-01-05,2007-01-08,AMAT,Legal/Regulatory,Applied Materials' Review under the HSR Antitr...,0.0,NaN,NaN,1,2007-01-06,337
1,148666,2007-01-05,2007-01-05,FCS,Legal/Regulatory,Fairchild Semiconductor Appeals Ruling in ZTE ...,8.4,$M,Zhongxing Telecom Ltd,1,2007-01-06,20486
2,77994,2007-01-09,2007-01-09,XLNX,Legal/Regulatory,Xilinx announces dismissal of shareholder deri...,0.0,NaN,NaN,1,2007-01-10,8344


Let's go over the columns:
- **event_id**: the unique identifier for this event.
- **asof_date**: EventVestor's timestamp of event capture.
- **trade_date**: for event announcements made before trading ends, trade_date is the same as event_date. For announcements issued after market close, trade_date is next market open day.
- **symbol**: stock ticker symbol of the affected company.
- **event_type**: this should always be *Legal/Regulatory*.
- **event_headline**: a brief description of the event
- **legal_amount**: amount mentioned in the case, if any.
- **legal_units**: units of the legal_amount: most commonly millions of dollars.
- **legal_entity**: the related entity in the legal case.
- **event_rating**: this is always 1. The meaning of this is uncertain.
- **timestamp**: this is our timestamp on when we registered the data.
- **sid**: the equity's unique identifier. Use this instead of the symbol.

We've done much of the data processing for you. Fields like `timestamp` and `sid` are standardized across all our Store Datasets, so the datasets are easy to combine. We have standardized the `sid` across all our equity databases.

We can select columns and rows with ease. Below, we'll fetch all 2014 events involving General Motors.

In [6]:
# get GM's sid first
gm_sid = symbols('GM').sid
cases = legal_and_regulatory[('2013-12-31' < legal_and_regulatory['asof_date']) & 
                        (legal_and_regulatory['asof_date'] <'2015-01-01') & 
                        (legal_and_regulatory.sid == gm_sid)]
# When displaying a Blaze Data Object, the printout is automatically truncated to ten rows.
cases.sort('asof_date')

,event_id,asof_date,trade_date,symbol,event_type,event_headline,legal_amount,legal_units,legal_entity,event_rating,timestamp,sid
0,1695334,2014-03-19,2014-03-20,GM,Legal/Regulatory,General Motors Co. Faces Class Action Lawsuit ...,0,NaN,Hagens Berman,1,2014-03-20,40430
1,1696156,2014-03-21,2014-03-21,GM,Legal/Regulatory,General Motors Co. Faces Class Action Lawsuit,0,NaN,Pomerantz LLP,1,2014-03-22,40430
2,1703401,2014-04-22,2014-04-23,GM,Legal/Regulatory,General Motors Company Faces Class Action Laws...,0,NaN,Law Offices of Howard G. Smith; Glancy Binkow ...,1,2014-04-23,40430
3,1725393,2014-05-16,2014-05-19,GM,Legal/Regulatory,General Motors Co. Faces Class Action Lawsuits,0,NaN,Pomerantz LLP,1,2014-05-17,40430
4,1725455,2014-05-17,2014-05-19,GM,Legal/Regulatory,General Motors to Pay $35M Fine Over Delay to ...,35,$M,NaN,1,2014-05-18,40430
5,1736421,2014-06-18,2014-06-18,GM,Legal/Regulatory,General Motors Co. Faces Class Action Lawsuit,0,NaN,Hagens Berman Sobol Shapiro,1,2014-06-19,40430
6,1768189,2014-08-09,2014-08-09,GM,Legal/Regulatory,Judge Rejects General Motors Co. Motion to Dis...,0,NaN,NaN,1,2014-08-10,40430


Now suppose we want a DataFrame of the Blaze Data Object above, but only want entries with a non-zero legal_amount. Further, we want to drop the `event_rating` and `event_type`.

In [7]:
df = odo(cases, pd.DataFrame)
df = df[df.legal_amount > 0]
df.drop(df[['event_type','event_rating']], axis=1, inplace=True)
df

,event_id,asof_date,trade_date,symbol,event_headline,legal_amount,legal_units,legal_entity,timestamp,sid
4,1725455,2014-05-17,2014-05-19,GM,General Motors to Pay $35M Fine Over Delay to ...,35,$M,NaN,2014-05-18,40430
